# 🔥 Fire Extinguisher Detection - YOLOv8 Training
## For Theta Edge Berhad HSE Inspection Platform

This notebook trains a YOLOv8 model to detect:
- Shell
- Hose
- Nozzle
- Safety Pin
- Pressure Gauge
- Pin Seal
- Service Tag

**Important:** Make sure to run this on a GPU runtime!
- Runtime → Change runtime type → Hardware accelerator: **GPU**

## 📦 Step 1: Install Dependencies

In [ ]:
# Install Ultralytics (YOLOv8)
!pip install ultralytics roboflow -q

# Import libraries
from ultralytics import YOLO
import os
from roboflow import Roboflow

print("✅ Dependencies installed successfully!")

## 📥 Step 2: Download Dataset from Roboflow

In [ ]:
# Download your fire extinguisher dataset
# IMPORTANT: Replace 'xxx' with your actual Roboflow API key

rf = Roboflow(api_key="xxx")  # ⚠️ REPLACE WITH YOUR ACTUAL API KEY
project = rf.workspace("fire-extinguisher-detection").project("fire-extinguisher-sz0p7")
version = project.version(3)
dataset = version.download("yolov8")

print(f"\n✅ Dataset downloaded to: {dataset.location}")
print(f"📊 Dataset YAML: {dataset.location}/data.yaml")

## 🔍 Step 3: Verify Dataset

In [ ]:
# Check dataset structure
import yaml

with open(f'{dataset.location}/data.yaml', 'r') as f:
    data_config = yaml.safe_load(f)

print("📋 Dataset Configuration:")
print(f"   Train images: {data_config.get('train', 'N/A')}")
print(f"   Validation images: {data_config.get('val', 'N/A')}")
print(f"   Test images: {data_config.get('test', 'N/A')}")
print(f"\n🏷️  Classes ({data_config.get('nc', 0)}):")
for i, cls in enumerate(data_config.get('names', [])):
    print(f"   {i}: {cls}")

## 🚀 Step 4: Train YOLOv8 Model

**Training Parameters:**
- Model: YOLOv8n (nano - fastest)
- Epochs: 100 (adjust if needed)
- Image size: 640x640
- Batch size: 16 (auto-adjusts based on GPU)

**This will take 20-40 minutes depending on your dataset size.**

In [ ]:
# Load a pretrained YOLOv8 model
model = YOLO('yolov8n.pt')  # Use 'yolov8s.pt' for more accuracy (slower)

# Train the model
results = model.train(
    data=f'{dataset.location}/data.yaml',
    epochs=100,              # Number of training epochs
    imgsz=640,               # Image size
    batch=16,                # Batch size (auto-adjusts)
    name='fire_extinguisher_v3',  # Experiment name
    patience=20,             # Early stopping patience
    save=True,               # Save checkpoints
    device=0,                # Use GPU 0
    project='runs/detect',   # Project directory
    exist_ok=True,           # Allow overwriting
    pretrained=True,         # Use pretrained weights
    optimizer='auto',        # Optimizer
    verbose=True,            # Verbose output
    seed=42,                 # Random seed for reproducibility
    plots=True,              # Generate training plots
)

print("\n✅ Training complete!")
print(f"📁 Results saved to: runs/detect/fire_extinguisher_v3")

## 📊 Step 5: View Training Results

In [ ]:
# Display training curves
from IPython.display import Image, display

print("📈 Training Results:")
display(Image(filename='runs/detect/fire_extinguisher_v3/results.png', width=800))

print("\n🎯 Confusion Matrix:")
display(Image(filename='runs/detect/fire_extinguisher_v3/confusion_matrix.png', width=600))

print("\n📸 Sample Predictions:")
display(Image(filename='runs/detect/fire_extinguisher_v3/val_batch0_pred.jpg', width=800))

## ✅ Step 6: Validate Model Performance

In [ ]:
# Load the best trained model
best_model = YOLO('runs/detect/fire_extinguisher_v3/weights/best.pt')

# Run validation
metrics = best_model.val()

print("\n📊 Model Performance:")
print(f"   mAP50: {metrics.box.map50:.3f}")
print(f"   mAP50-95: {metrics.box.map:.3f}")
print(f"   Precision: {metrics.box.mp:.3f}")
print(f"   Recall: {metrics.box.mr:.3f}")

print("\n🏷️  Per-Class Performance:")
for i, name in enumerate(data_config.get('names', [])):
    if i < len(metrics.box.ap50):
        print(f"   {name}: {metrics.box.ap50[i]:.3f}")

## 🧪 Step 7: Test on a Sample Image

In [ ]:
# Test on a validation image
import glob

# Get a random validation image
val_images = glob.glob(f'{dataset.location}/valid/images/*')
test_image = val_images[0] if val_images else None

if test_image:
    # Run prediction
    results = best_model.predict(test_image, conf=0.5, save=True)
    
    print("🔍 Detection Results:")
    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            cls_name = data_config['names'][cls_id]
            print(f"   ✓ {cls_name}: {conf:.2%} confidence")
    
    # Display result
    display(Image(filename=results[0].save_dir + '/' + os.path.basename(test_image), width=600))
else:
    print("❌ No validation images found")

## 💾 Step 8: Export Model for Deployment

Export to ONNX format for faster inference on your server.

In [ ]:
# Export to ONNX format (optional - for optimization)
onnx_model = best_model.export(format='onnx', simplify=True)
print(f"✅ ONNX model exported to: {onnx_model}")

# Also keep the PyTorch format for flexibility
print(f"\n📦 PyTorch model (.pt): runs/detect/fire_extinguisher_v3/weights/best.pt")

## 📥 Step 9: Download Model Weights

Download these files to your computer:
1. `best.pt` - Main model file (required)
2. `results.png` - Training metrics
3. `confusion_matrix.png` - Performance analysis

In [ ]:
# Zip the model weights and results
!zip -r fire_extinguisher_model_v3.zip runs/detect/fire_extinguisher_v3/weights

print("✅ Model packaged!")
print("\n📥 Download Instructions:")
print("   1. In the left sidebar, click the folder icon")
print("   2. Find 'fire_extinguisher_model_v3.zip'")
print("   3. Right-click → Download")
print("\n   OR run the cell below to download directly:")

In [ ]:
# Direct download (works in Colab)
from google.colab import files
files.download('fire_extinguisher_model_v3.zip')

# Also download individual files
files.download('runs/detect/fire_extinguisher_v3/weights/best.pt')
files.download('runs/detect/fire_extinguisher_v3/results.png')

## 🎉 Training Complete!

### Next Steps:
1. ✅ Download `best.pt` to your computer
2. 📁 Place it in your project: `pwa-inspection/models/best.pt`
3. 🚀 Deploy using `model_server.py`
4. ⚙️ Update your `.env.local` configuration

### Model Performance Guidelines:
- **mAP50 > 0.85**: Excellent ✅
- **mAP50 > 0.70**: Good ✓
- **mAP50 < 0.70**: Needs more training data or tuning ⚠️

### If Accuracy is Low:
- Add more training images (aim for 300+)
- Increase epochs to 150-200
- Use a larger model: `yolov8s.pt` or `yolov8m.pt`
- Check for annotation errors in Roboflow